In [ ]:
# ============================ IBM Quantum Runtime — Minimal Auth & Backend Discovery Template ============================
# Purpose:
#   - Save a user's IBM Quantum API key and instance CRN.
#   - Initialize QiskitRuntimeService.
#   - List available backends and pick one with basic safeguards.
# Notes:
#   - Replace the placeholders below with your own values.
#   - This template intentionally omits any algorithm/problem-specific code.
#   - Tested with Python 3.12.10 (should work with any 3.11.XX+) and Qiskit v2.0.0 on September 22, 2025.

import os
import traceback
from typing import List, Optional

try:
    # Core Runtime client
    from qiskit_ibm_runtime import QiskitRuntimeService, Options
except Exception as _imp_err:
    raise SystemExit(
        "[Setup] qiskit-ibm-runtime is not installed. "
        "Install with: pip install qiskit-ibm-runtime"
    ) from _imp_err

# ----------------------------- User-provided credentials (PLACEHOLDERS) -----------------------------
CHANNEL      = "ibm_quantum_platform"         # Leave as-is for IBM Quantum Platform
API_KEY      = "YOUR_IBM_QUANTUM_API_KEY"     # e.g., "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
INSTANCE_CRN = "YOUR_INSTANCE_CRN"            # e.g., "crn:v1:bluemix:public:quantum-computing:..."

# ----------------------------- Utility: fail-safe printer -----------------------------
def _fail(msg: str, exc: Optional[BaseException] = None) -> None:
    """Print a clear error message plus traceback (if available)."""
    print(f"[IBM Runtime] {msg}")
    if exc is not None:
        print("Details:", repr(exc))
        tb = traceback.format_exc()
        if tb:
            print(tb)

# ----------------------------- 1) Save credentials to local config -----------------------------
try:
    QiskitRuntimeService.save_account(
        channel=CHANNEL,
        token=API_KEY,
        instance=INSTANCE_CRN,
        overwrite=True,  
    )
    print(f"✓ Account saved (channel={CHANNEL}, instance=CRN, token={API_KEY[:4]}…{API_KEY[-4:] if len(API_KEY) >= 8 else '****'}).")
except Exception as e:
    _fail("Could not save account. Verify CHANNEL/API key/CRN.", e)
    raise SystemExit

# ----------------------------- 2) Initialize service and enumerate backends -----------------------------
try:
    service = QiskitRuntimeService()  # picks up saved credentials
    backends = service.backends(instance=INSTANCE_CRN)
    if not backends:
        print("! No backends visible under this CRN (permissions/plan/region?).")
    else:
        names = [b.name for b in backends]
        print(f"✓ {len(names)} backend(s) visible on this instance. Examples: {names[:5]}")
except Exception as e:
    _fail("Unable to initialize service or list backends for the given CRN.", e)
    raise SystemExit

# ----------------------------- 3) Choose a backend with simple, safe heuristics -----------------------------
PREFERRED: List[str] = [
    # Optional: put any preferred backend names here; the code falls back automatically.
    # e.g., "ibm_torino", "ibm_brisbane"
]

def _pick_backend(service: QiskitRuntimeService, instance: str, preferred_names: List[str]):
    """Pick a backend, preferring known names, else first permitted & operational with the smallest queue."""
    # Try preferred names first
    for name in preferred_names:
        try:
            return service.backend(name=name, instance=instance)
        except Exception:
            pass

    # Otherwise, choose a reasonable default
    try:
        candidates = service.backends(instance=instance, simulator=False, operational=True)
        if not candidates:
            candidates = service.backends(instance=instance)
        if not candidates:
            return None

        scored = []
        for b in candidates:
            try:
                st = b.status()
                q = getattr(st, "pending_jobs", None)
                score = q if isinstance(q, int) else 10**9
            except Exception:
                score = 10**9
            scored.append((score, b))
        scored.sort(key=lambda t: t[0])
        return scored[0][1]
    except Exception:
        return None

backend = _pick_backend(service, INSTANCE_CRN, PREFERRED)

if backend is None:
    _fail("Could not access a permitted backend under your CRN. Check entitlements/plan/region, or try a different instance.")
    raise SystemExit

print(f"✓ Selected backend: {backend.name}  [instance: CRN]")